# 3. Etiquetado de Secuencias: Parte 3

## 3.1. Corpus de Entrenamiento

In [1]:
tagged_sents = [
    list(zip('El gato come pescado .'.split(),
         'D N V N P'.split())),
    list(zip('La gata come salmón .'.split(),
         'D N V N P'.split())),
]

In [2]:
tagged_sents

[[('El', 'D'), ('gato', 'N'), ('come', 'V'), ('pescado', 'N'), ('.', 'P')],
 [('La', 'D'), ('gata', 'N'), ('come', 'V'), ('salmón', 'N'), ('.', 'P')]]

## 3.2. Extracción de Features

Primero definimos la función que extrae los features para cada palabra en forma de diccionario:

In [3]:
def features(word):
    return {
        'lower': word.lower(),
        'istitle': word.istitle(),
    }

Ahora convertimos el corpus dos listas con las instancias X y sus etiquetas y_true:

In [4]:
X = []
y_true = []
for sent in tagged_sents:
    for word, tag in sent:
        x = features(word)
        X.append(x)
        y_true.append(tag)

In [5]:
list(zip(X, y_true))

[({'lower': 'el', 'istitle': True}, 'D'),
 ({'lower': 'gato', 'istitle': False}, 'N'),
 ({'lower': 'come', 'istitle': False}, 'V'),
 ({'lower': 'pescado', 'istitle': False}, 'N'),
 ({'lower': '.', 'istitle': False}, 'P'),
 ({'lower': 'la', 'istitle': True}, 'D'),
 ({'lower': 'gata', 'istitle': False}, 'N'),
 ({'lower': 'come', 'istitle': False}, 'V'),
 ({'lower': 'salmón', 'istitle': False}, 'N'),
 ({'lower': '.', 'istitle': False}, 'P')]

## 3.3 Vectorización

Usaremos DictVectorizer de scikit-learn para vectorizar.

Primero instanciamos el vectorizador:

In [6]:
from sklearn.feature_extraction import DictVectorizer

vect = DictVectorizer()

Luego lo "entrenamos". El entrenamiento sólo define un mapeo entre los features y las columnas de los vectores de salida:

In [7]:
vect.fit(X)

DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
               sparse=True)

In [8]:
vect.get_feature_names()

['istitle',
 'lower=.',
 'lower=come',
 'lower=el',
 'lower=gata',
 'lower=gato',
 'lower=la',
 'lower=pescado',
 'lower=salmón']

Luego podemos usarlo para vectorizar:

In [9]:
v = vect.transform(X[0])
v

<1x9 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

Los vectores resultantes son "sparse" porque en su mayoría contienen ceros.

In [10]:
v.toarray()

array([[1., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [11]:
vect.transform({'lower': 'gato', 'istitle': True}).toarray()

array([[1., 0., 0., 0., 0., 1., 0., 0., 0.]])

## 3.4. Clasificación

Usaremos el clasificador MultinomialNB. Primero instanciamos:

In [12]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

Luego vectorizamos y entrenamos:

In [13]:
X2 = vect.transform(X)
clf.fit(X2, y_true)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Ahora el clasificador está listo para predecir:

In [14]:
x = {'lower': 'el', 'istitle': True}
x2 = vect.transform(x)
clf.predict(x2)

array(['D'], dtype='<U1')

Podemos así etiquetar una nueva oración:

In [15]:
sent = 'Lx gatx come melón .'.split()

X_test = [features(word) for word in sent]
X2_test = vect.transform(X_test)
y_pred = clf.predict(X2_test)
y_pred

array(['D', 'N', 'V', 'N', 'P'], dtype='<U1')

**Pregunta:** ¿Porqué clasificó bien las palabras nuevas?

## 3.5. Pipelines

Podemos usar pipelines para secuenciar componentes. Esto permite entrenarlos y usarlos juntos:

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('vect', DictVectorizer()),
    ('clf', MultinomialNB())
])

Entrenamos:

In [17]:
pipeline.fit(X, y_true);

Y lo usamos:

In [18]:
sent = 'Lx gatx come melón .'.split()

X_test = [features(word) for word in sent]
y_pred = pipeline.predict(X_test)
y_pred

array(['D', 'N', 'V', 'N', 'P'], dtype='<U1')